In [1]:
# Model Performance Analysis and Explainability

# Cell 1: Setup
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# Setup paths
if 'notebooks' in str(Path.cwd()):
    PROJECT_ROOT = Path.cwd().parent
else:
    PROJECT_ROOT = Path.cwd()

sys.path.insert(0, str(PROJECT_ROOT / 'src'))

from utils.metrics import RegressionMetrics

print("✓ Imports successful")
print(f"Project root: {PROJECT_ROOT}")

✓ Imports successful
Project root: d:\Dev\Projects\qrucible


In [2]:
# Cell 2: Load Trained Models
print("\n" + "="*70)
print("LOADING TRAINED MODELS")
print("="*70)

models_dir = PROJECT_ROOT / 'results' / 'models'

# Check available models
available_models = list(models_dir.glob('*.pkl'))
print(f"\nFound {len(available_models)} trained models:")
for model_path in available_models:
    print(f"  - {model_path.name}")

# Load models
loaded_models = {}

for model_path in available_models:
    model_name = model_path.stem
    try:
        model_data = joblib.load(model_path)
        loaded_models[model_name] = model_data
        print(f"\n✓ Loaded: {model_name}")
    except Exception as e:
        print(f"\n✗ Failed to load {model_name}: {e}")



LOADING TRAINED MODELS

Found 0 trained models:
